<h1 style='text-align: center'>Week 3 Assignment, Applied Data Science Capstone</h1>
<h2 style='text-align: center'>Segmenting and Clustering Neighborhoods in Toronto</h2>

In [2]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import requests

print('Libraries imported.')

SyntaxError: invalid syntax (<ipython-input-2-469557a116c8>, line 5)

<h3> Data read-in from Wikipedia </h3>

In [61]:
src_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response = requests.get(src_url)

df_raw = pd.read_html(response.text)[0]
df_raw.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3> Data cleaning - the raw data is kept, just in case. </h3>

In [62]:
df_proc = df_raw.copy()

Rename the columns appropriately

In [63]:
df_proc.columns = ['PostalCode', 'Borough', 'Neighborhood']

Drop any boroughs which are not assigned:

In [64]:
df_proc = df_proc[df_proc['Borough']!='Not assigned'].reset_index().drop('index', axis=1)

Rename any neighborhoods that are not assigned to their respective borough:

In [65]:
df_proc['Neighborhood'].replace('Not assigned', df_proc['Borough'], inplace=True)

Group the dataframe by postal code to sum up the neighborhoods:

In [66]:
df_proc = df_proc.groupby(by='PostalCode').sum().reset_index()

<h3>Take a look at the resulting dataframe:</h3>

In [67]:
df_proc.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [68]:
df_proc.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M6R,North York,Downsview
freq,1,24,4


In [69]:
df_proc.shape

(103, 3)

<h3> The dataframe is now cleaned and ready for further evaluation. </h3>
<h3> The next step is assigning latitudal and longitudinal coordinates for each postal code using Geocoder. </h3>

<h3> Define a function for getting the required values. </h3>

In [72]:
def get_latlng(postal_codes):
    '''
    Returns a list each for latitude and longitude by entering the postal codes of boroughs in Toronto, Ontario, CA.
    '''
    latitudes = []
    longitudes = []
    geolocator = Nominatim(user_agent='toronto_explorer')
    
    for code in postal_codes:
        print('Start Loop')
        location = None
        while location == None: # For timeout requests or invalid API calls
            print(location)
            location = geolocator.geocode('{}, Toronto, Ontario'.format(code))
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
        print('End loop')
    return latitudes, longitudes

In [73]:
get_latlng(df_proc['PostalCode'])

Start Loop
None
End loop
Start Loop
None
End loop
Start Loop
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [55]:
get_latlng(df_proc['PostalCode'])

Toronto, Golden Horseshoe, Ontario, Canada
M1B
Toronto, Golden Horseshoe, Ontario, Canada
M1C
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [59]:
url = 'https://maps.googleapis.com/maps/api/geocode/json?address=24%20Sussex%20Drive%20Ottawa%20ON'
list(requests.get(url))

[b'{\n   "error_message" : "You must use an API key to authenticate each request to Google Maps Platform APIs. For additional inform',
 b'ation, please refer to http://g.co/dev/maps-no-account",\n   "results" : [],\n   "status" : "REQUEST_DENIED"\n}\n']